In [1]:
import pandas as pd
import numpy as np
from PIL import Image 
import matplotlib.pyplot as plt
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.models as models
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import cv2

In [2]:
Thickness = pd.read_csv('./data/Thickness.csv')
Thickness = Thickness.drop([33]) # 9-1 drop
Thickness.reset_index(drop=True, inplace=True)
Thickness

,Thickness
0,48.266667
1,47.966667
2,48.166667
3,48.600000
4,59.533333
...,...
122,84.633333
123,98.366667
124,97.033333
125,96.266667


In [3]:
img_path_list = []
Top_path = './data/box_img/Top/'
Bot_path = './data/box_img/Bot/'

for i in range(1, 33):
    for j in range(1, 5):
        if i == 9 and j == 1: 
            continue
        img_path_list.append('./data/test_img/AI-{}_0{}_transformed.jpg'.format(i, j))

색상 변환시 32float으로 하면 오차가 줄어듦

In [4]:
# 첫번째 bot 이미지를 기준 이미지로 설정
reference_bot = cv2.cvtColor(np.array(Image.open(Bot_path + 'Bot_0.jpg'), dtype=np.float32), cv2.COLOR_RGB2BGR)
reference_bot_hsv = cv2.cvtColor(reference_bot, cv2.COLOR_BGR2HSV)


In [8]:
Top_change_imp_VS_path = './data/box_img/Top_change_VS_imp/'

# 폴더가 존재하지 않으면 생성
if not os.path.exists(Top_change_imp_VS_path):
    os.makedirs(Top_change_imp_VS_path)

for i in range(len(img_path_list)):
    # 이미지 로드 후 데이터 타입을 float32로 변환
    top = cv2.cvtColor(np.array(Image.open(Top_path + 'Top_{}.jpg'.format(i)), dtype=np.float32), cv2.COLOR_RGB2BGR)
    bot = cv2.cvtColor(np.array(Image.open(Bot_path + 'Bot_{}.jpg'.format(i)), dtype=np.float32), cv2.COLOR_RGB2BGR)

    # 명도와 채도의 변화를 백분율로 계산
    bot_hsv = cv2.cvtColor(bot, cv2.COLOR_BGR2HSV)
    delta_V = bot_hsv[:,:,2].mean() / reference_bot_hsv[:,:,2].mean()
    delta_S = bot_hsv[:,:,1].mean() / reference_bot_hsv[:,:,1].mean()

    print(f"{i}번째 delta \n delta_V: {delta_V} delta_S:{delta_S}")
    print(f"bot V mean : \n bot: {bot_hsv[:,:,2].mean()} , ref: {reference_bot_hsv[:,:,2].mean()}")
    print(f"bot S mean : \n bot: {bot_hsv[:,:,1].mean()} , ref: {reference_bot_hsv[:,:,1].mean()} \n")

    # 계산한 변화를 top 이미지에 적용
    top_hsv = cv2.cvtColor(top, cv2.COLOR_BGR2HSV)
    print(f"prev top : \n v: {top_hsv[:,:,2].mean()}  s: {top_hsv[:,:,1].mean()}")
    top_hsv[:,:,2] = np.clip(top_hsv[:,:,2] * delta_V, 0, 255) 
    top_hsv[:,:,1] = np.clip(top_hsv[:,:,1] * delta_S, 0, 255)
    print(f"changed_top_hsv : \n v: {top_hsv[:,:,2].mean()}, s: {top_hsv[:,:,1].mean()} \n")

    top_changed = cv2.cvtColor(top_hsv, cv2.COLOR_HSV2BGR)
    print(f"prev_RGB : \n R: {top[:,:,2].mean()} G: {top[:,:,1].mean()} B: {top[:,:,0].mean()}")
    print(f"saved_RGB : \n R: {top_changed[:,:,2].mean()} G: {top_changed[:,:,1].mean()} B: {top_changed[:,:,0].mean()}")
    print(f"==============================================================================")

    Image.fromarray(cv2.cvtColor(top_changed, cv2.COLOR_BGR2RGB).astype(np.uint8)).save(os.path.join(Top_change_imp_VS_path, 'Top_change_VS_{}.jpg'.format(i)))


0번째 delta 
 delta_V: 1.0 delta_S:1.0
bot V mean : 
 bot: 50.446800231933594 , ref: 50.446800231933594
bot S mean : 
 bot: 0.401544988155365 , ref: 0.401544988155365 

prev top : 
 v: 11.576000213623047  s: 0.3021555244922638
changed_top_hsv : 
 v: 11.576000213623047, s: 0.3021555244922638 

prev_RGB : 
 R: 11.576000213623047 G: 10.883199691772461 B: 8.13640022277832
saved_RGB : 
 R: 11.576000213623047 G: 10.883199691772461 B: 8.13640022277832
1번째 delta 
 delta_V: 0.8532236218452454 delta_S:0.9823868274688721
bot V mean : 
 bot: 43.04240036010742 , ref: 50.446800231933594
bot S mean : 
 bot: 0.39447250962257385 , ref: 0.401544988155365 

prev top : 
 v: 12.741600036621094  s: 0.13359972834587097
changed_top_hsv : 
 v: 10.871434211730957, s: 0.13124661147594452 

prev_RGB : 
 R: 12.471199989318848 G: 12.624799728393555 B: 11.213600158691406
saved_RGB : 
 R: 10.64478588104248 G: 10.77353286743164 B: 9.59067153930664
2번째 delta 
 delta_V: 0.8291427493095398 delta_S:0.625480592250824
bot V m

개선된 전처리 데이터 보기

In [6]:
top_avg_bgr = []
top_avg_hsv = []

for i in range(len(img_path_list)):
    # 이미지 로드
    img = cv2.cvtColor(np.array(Image.open(Top_change_imp_VS_path + 'Top_change_VS_{}.jpg'.format(i))), cv2.COLOR_RGB2BGR)
    # RGB 평균 계산
    avg_bgr = img.mean(axis=(0,1))
    top_avg_bgr.append(avg_bgr)
    # HSV 평균 계산
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    avg_hsv = img_hsv.mean(axis=(0,1))
    top_avg_hsv.append(avg_hsv)

Table_dataset = pd.DataFrame({'Top_R' : [i[2] for i in top_avg_bgr], 
                              'Top_G' : [i[1] for i in top_avg_bgr], 
                              'Top_B' : [i[0] for i in top_avg_bgr],
                              'Top_H' : [i[0] for i in top_avg_hsv], 
                              'Top_S' : [i[1] for i in top_avg_hsv], 
                              'Top_V' : [i[2] for i in top_avg_hsv],
                              'Thickness': Thickness['Thickness']})

Table_dataset

,Top_R,Top_G,Top_B,Top_H,Top_S,Top_V,Thickness
0,11.5752,10.8824,8.0332,22.1580,79.0864,11.5752,48.266667
1,10.2804,10.4340,9.0232,23.5584,41.5664,10.5508,47.966667
2,10.0228,9.9236,7.9204,29.0352,56.4348,10.0308,48.166667
3,8.3744,8.5136,6.1348,26.7236,77.8136,8.6560,48.600000
4,4.1716,2.2884,1.1488,27.9292,211.7900,4.3000,59.533333
...,...,...,...,...,...,...,...
122,1.8884,5.0128,18.5152,114.3780,229.4796,18.5152,84.633333
123,6.9784,15.3760,26.5632,106.8272,188.4016,26.5632,98.366667
124,3.3400,13.3692,33.3720,109.8596,230.0004,33.3720,97.033333
125,5.8372,15.0308,31.7036,109.2272,208.5904,31.7036,96.266667


In [ ]:
Table_dataset.to_csv('./data/Top_change_imp_VS_path.csv', index=False)
